In [ ]:
# Install AutoGluon
!pip install autogluon

import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.preprocessing import StandardScaler

directory = '../input/tabular-playground-series-aug-2021/'
label = 'loss'
eval_metric = 'root_mean_squared_error'
problem_type = 'regression'
save_path = './'
time_limit = 30 * 60

# Load data
train_df = pd.read_csv(directory+'train.csv')
test_df = pd.read_csv(directory+'test.csv')
sample_submission = pd.read_csv(directory+'sample_submission.csv')

# Drop the id columns
train_df = train_df.drop(['id'], axis=1)
test_df = test_df.drop(['id'], axis=1)

# Standard scale values
scaler = StandardScaler()
train_df.loc[:, train_df.columns != 'loss'] = scaler.fit_transform(train_df.loc[:, train_df.columns != 'loss'])
test_df = pd.DataFrame(scaler.transform(test_df), columns=train_df.loc[:, train_df.columns != 'loss'].columns)

# Fit models
predictor = TabularPredictor(label=label, eval_metric=eval_metric, problem_type=problem_type).fit(
    train_df, auto_stack=True, presets='best_quality', time_limit=time_limit,
)

results = predictor.fit_summary()
print(results)

# Predict
y_pred = predictor.predict(test_df)

# Save
sample_submission['loss'] = y_pred
sample_submission.to_csv('submission.csv', index=False)